In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        ...

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import random

In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers



In [61]:
categories = ['NORMAL','PNEUMONIA']
data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
data_val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'

In [62]:
img_data_val_array = []
for category in categories:
    path = os.path.join(data_val_dir,category)
    for img in os.listdir(path):
        class_num =categories.index(category) 
        img_ = os.path.join(path,img)
        img_array = cv2.imread(img_,cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array,(150,150))
        img_data_val_array.append([img_array,class_num])


In [44]:
img_data_array = []
for category in categories:
    path = os.path.join(data_dir,category)
    for img in os.listdir(path):
        class_num =categories.index(category) 
        img_ = os.path.join(path,img)
        img_array = cv2.imread(img_,cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array,(150,150))
        img_data_array.append([img_array,class_num])


In [45]:
random.shuffle(img_data_array)

In [66]:
#splitting Feature and lables 
X = []
y = []
for feature,label in img_data_array:
    X.append(feature)
    y.append(label)
X = np.array(X).reshape(-1,150,150)
y = np.array(y)

X_val = []
y_val = []
for feature,label in img_data_val_array:
    X_val.append(feature)
    y_val.append(label)
X_val = np.array(X_val).reshape(-1,150,150)
y_val = np.array(y_val)


In [67]:
#Normalizing the data
X = X/255
X_val = X_val/255

In [68]:
model = Sequential([
    
    Conv2D(64,(4,4),input_shape = (150,150,1),activation = 'relu'),
    MaxPooling2D(pool_size = (3,3)),
    Conv2D(64,(4,4),activation = 'relu'),
    MaxPooling2D(pool_size = (3,3)),
    Conv2D(64,(4,4),activation = 'relu'),
    MaxPooling2D(pool_size = (3,3)),
    Flatten(),
    Dense(128,activation = 'relu',kernel_regularizer=regularizers.L2(0.001)),
    Dense(128,activation = 'relu',kernel_regularizer=regularizers.L2(0.0001)),
    Dense(1,activation = 'sigmoid')
    
])



In [71]:
metrics = [
    tf.keras.metrics.Recall(name = 'recall'),
    tf.keras.metrics.Precision(name = 'precision')
]
model.compile(
    loss = 'binary_crossentropy',
    optimizer = Adam(learning_rate = .001),
    metrics = metrics
)

In [74]:
class_weight = {0: 1, 1: 5}
model.fit(X,y,epochs = 5,class_weight = class_weight,validation_data = (X_val,y_val))


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 182s 1s/step - loss: 1.2320 - precision: 0.7416 - recall: 0.9664 - val_loss: 0.2452 - val_precision: 0.9449 - val_recall: 0.9701
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - loss: 0.4098 - precision: 0.8876 - recall: 0.9885 - val_loss: 0.1513 - val_precision: 0.9586 - val_recall: 0.9871
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - loss: 0.2900 - precision: 0.9312 - recall: 0.9881 - val_loss: 0.1351 - val_precision: 0.9558 - val_recall: 0.9933
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 192s 1s/step - loss: 0.2147 - precision: 0.9505 - recall: 0.9922 - val_loss: 0.1635 - val_precision: 0.9391 - val_recall: 0.9985
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - loss: 0.1848 - precision: 0.9554 - recall: 0.9933 - val_loss: 0.4219 - val_precision: 0.8386 - val_recall: 1.0000


In [75]:
model.save("pneumonia_model.h5")